In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
#import scipy.stats as linreg
import numpy as np
import requests
import json
import hvplot.pandas

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")


# Import the API key
from config import geoapify_key

In [12]:
# Set the search parameters
target_city = "Kyoto, Yapan"

params = {
    "text": target_city,
    "apiKey": geoapify_key
}

# Build URL using the geocode endpoint
base_url = "https://api.geoapify.com/v1/geocode/search"

In [13]:
# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
#print(json.dumps(response, indent=4, sort_keys=True))

In [14]:
 # Extract lat/lon
lat = response["features"][0]["properties"]["lat"]
lon = response["features"][0]["properties"]["lon"]

# Print results
print(f"{target_city}: {lat}, {lon}")

Kyoto, Yapan: 35.021041, 135.7556075


Finding 500 restaurants within 10 km from the center of Kyoto

In [15]:
# Set the geographical coordinates for the Sydney Opera House
latitude = lat
longitude = lon

# Set the parameters for the type of place
categories = "catering.restaurant"
radius = 10000

# Set the parameters for the type of search
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
limit = 500

# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Run request
response = requests.get(base_url, params=params).json()

# Print the json (pretty printed)
print(json.dumps(response, indent=4, sort_keys=True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    135.7572641,
                    35.022655299653195
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "\u00c0 la Chalamont",
                "address_line2": "Shimochojamachi Dori, Kamigyo Ward, Kyoto, 602-8004, Japan",
                "categories": [
                    "catering",
                    "catering.restaurant",
                    "catering.restaurant.french"
                ],
                "city": "Kyoto",
                "country": "Japan",
                "country_code": "jp",
                "county": "Kyoto Prefecture",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "addr:city": "\u4eac\u90fd\u5e02",
                  

In [16]:
#kyoto_rest=pd.DataFrame({'Name': [], 'Type': [], 'Address': [], 'Latitude':[], 'Magnitude':[]})
rest=[]
lati=[]
long=[]
add=[]
cuis=[]
       # cities.append(city)

for r in range(0, len(response["features"])):
        try:
                lati=response["features"][r]['properties']['lat']
                long=response["features"][r]['properties']['lon']
                add.append(response["features"][r]['properties']['address_line2'])
                rest.append(response["features"][r]['properties']['place_id'])
                cuis.append(response['features'][r]['properties']['datasource']['raw']["cuisine"])
                #rest.append(response['features'][r]['properties']['datasource']['raw']["name_en"])
        except:
        # Append null values
                cuis.append('')
        pass

        
kyoto_rest=pd.DataFrame({'Name': rest, 'Cuisine': cuis, 'Address': add, 'Latitude': lati, 'Longitude': long})

kyoto_rest.head()
#print(response["features"][5]['properties']['datasource']['raw']['cuisine'])


,Name,Cuisine,Address,Latitude,Longitude
0,51eafdeb813bf8604059b88b6d5ee6824140f00103f901...,french,"Shimochojamachi Dori, Kamigyo Ward, Kyoto, 602...",35.004458,135.770201
1,510e98874c39f8604059ec499c2166824140f00103f901...,italian,"出水通室町西入 1F Koromonotana street, Kamigyo Ward, ...",35.004458,135.770201
2,51954c4eed0cf8604059158742c6a3824140f00103f901...,,"堀川商店街, Kamigyo Ward, Kyoto, 602-8138, Japan",35.004458,135.770201
3,5155968d7340f860405913d8b0fc54824140f00103f901...,,"Muromachi street, Kamigyo Ward, Kyoto, 604-000...",35.004458,135.770201
4,51382cb24236f8604059793a32ac3d824140f00103f901...,,"Marutamachi-dori, Kamigyo Ward, Kyoto, 604-009...",35.004458,135.770201


In [17]:
#url = requests.get(base_url, params=params).url
#print(url)

In [18]:

# Configure the map
map_plot = kyoto_rest.hvplot.points(
    "Longitude",
    "Latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    color = "Cuisine",
    size=0.5
)

# Display the map plot
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Cuisine)